### Consigna del desafío 1

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**2**. Construir un modelo de clasificación por prototipos (tipo zero-shot). Clasificar los documentos de un conjunto de test comparando cada uno con todos los de entrenamiento y asignar la clase al label del documento del conjunto de entrenamiento con mayor similaridad.

**3**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**4**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.

#### Parte 1
#### Vectorizar y medir similaridad para 5 documentos al azar

In [1]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.datasets import fetch_20newsgroups

# 1) Cargar datos
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
y_train = newsgroups_train.target
target_names = newsgroups_train.target_names
docs = newsgroups_train.data

# 2) Vectorizar con TfidfVectorizer
tfidfvect = TfidfVectorizer()
X_train = tfidfvect.fit_transform(docs)

# 3) Seleccionar 5 documentos al azar (fijo semilla para reproducibilidad)
rng = np.random.default_rng(42)
n_docs = X_train.shape[0]
chosen_idxs = rng.choice(n_docs, size=5, replace=False)

# 4) Para cada documento elegido, medir similaridad coseno con todo el train
top_k = 6  # pedimos 6 para luego eliminar el mismo documento y quedarnos con 5 similares
for idx in chosen_idxs:
    print("==============================================================")
    print(f"Documento elegido (índice): {idx}")
    print(f"Etiqueta: {target_names[y_train[idx]]}")
    print("Fragmento del documento (primeros 400 caracteres):")
    print(docs[idx][:400].strip().replace("\n", " "))
    print("--------------------------------------------------------------")
    # calcular similaridad coseno entre este doc y todos
    cossim = cosine_similarity(X_train[idx], X_train)[0]
    # obtener índices ordenados de mayor a menor similaridad
    sorted_idxs = np.argsort(cossim)[::-1]
    # excluir el propio documento (aparece primero con similitud 1.0)
    most_similar = [i for i in sorted_idxs if i != idx][:5]
    print("5 documentos más similares (índice, etiqueta, puntaje, fragmento):")
    for sim_idx in most_similar:
        print(f"\n- Índice: {sim_idx}")
        print(f"  Etiqueta: {target_names[y_train[sim_idx]]}")
        print(f"  Similaridad (coseno): {cossim[sim_idx]:.4f}")
        snippet = docs[sim_idx][:300].strip().replace("\n", " ")
        print(f"  Fragmento: {snippet}")
    print("==============================================================\n")


Documento elegido (índice): 8754
Etiqueta: talk.religion.misc
Fragmento del documento (primeros 400 caracteres):
/(hudson) /If someone inflicts pain on themselves, whether they enjoy it or not, they /are hurting themselves.  They may be permanently damaging their body.  That is true.  It is also none of your business.    Some people may also reason that by reading the bible and being a Xtian you are permanently damaging your brain.  By your logic, it would be OK for them to come into your home, take away yo
--------------------------------------------------------------
5 documentos más similares (índice, etiqueta, puntaje, fragmento):

- Índice: 6552
  Etiqueta: talk.religion.misc
  Similaridad (coseno): 0.4904
  Fragmento: If I have a habit that I really want to break, and I am willing to make whatever sacrifice I need to make to break it, then I do so. There have been bad habits of mine that I've decided to put forth the effort to break, and I've done so; there have been other bad ha

Conclusion:

- En el doc 8754 (talk.religion.misc), casi todos los doc más similares también son talk.religion.misc,apareció un talk.politics.mideast, el texto es bastante similar a los otros con la diferencia que mezcla religión y política.

- En el doc 4965 (comp.sys.mac.hardware), los doc más similares son del mismo grupo (Mac hardware), aunque también aparecen algunos de PC hardware, lo cual también tiene sentido porque se habla del mismo tema "puertos de una impresora".

- En el doc 7404 (comp.os.ms-windows.misc), los doc más similares son de comp.windows.x, o sea, siguen siendo temas de sistemas de ventanas y aplicaciones.

- En el doc 1009 (talk.politics.guns), casi todos los doc más similares son también de talk.politics.guns, salvo un doc de alt.atheism, que trata el mismo tema pero relacionando la política y religión.

- En el doc 4899 (sci.crypt), varios de los doc más similares son sci.crypt, pero también aparecen talk.politics.mideast o alt.atheism, porque algunos textos tocan temas de política o sociedad en publicaciones.

#### Parte 2
#### Clasificación por prototipos (zero-shot style)

In [2]:
from sklearn.metrics import f1_score

# 1) Cargar datos train y test
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

X_train_texts, y_train = newsgroups_train.data, newsgroups_train.target
X_test_texts, y_test = newsgroups_test.data, newsgroups_test.target
target_names = newsgroups_train.target_names

# 2) Vectorizar con TF-IDF (entrenar solo en train)
tfidfvect = TfidfVectorizer()
X_train = tfidfvect.fit_transform(X_train_texts)
X_test = tfidfvect.transform(X_test_texts)

# 3) Clasificación por prototipos:
# para cada documento de test buscamos el train más similar
y_pred = []
for i in range(X_test.shape[0]):
    # similaridad con todos los documentos de train
    sims = cosine_similarity(X_test[i], X_train)[0]
    # índice del más similar
    best_idx = np.argmax(sims)
    # asignamos su etiqueta
    y_pred.append(y_train[best_idx])

y_pred = np.array(y_pred)

# 4) Evaluar desempeño con F1-score macro
f1_macro = f1_score(y_test, y_pred, average='macro')
print("F1-score (macro) del clasificador por prototipos:", f1_macro)

# Mostrar primeras 5 predicciones vs reales
for i in range(5):
    print("--------------------------------------------------")
    print("Texto test:")
    print(X_test_texts[i][:300].replace("\n"," "))
    print(f"\nEtiqueta real: {target_names[y_test[i]]}")
    print(f"Etiqueta predicha: {target_names[y_pred[i]]}")


F1-score (macro) del clasificador por prototipos: 0.5049911553681621
--------------------------------------------------
Texto test:
I am a little confused on all of the models of the 88-89 bonnevilles. I have heard of the LE SE LSE SSE SSEI. Could someone tell me the differences are far as features or performance. I am also curious to know what the book value is for prefereably the 89 model. And how much less than book value can

Etiqueta real: rec.autos
Etiqueta predicha: alt.atheism
--------------------------------------------------
Texto test:
I'm not familiar at all with the format of these "X-Face:" thingies, but after seeing them in some folks' headers, I've *got* to *see* them (and maybe make one of my own)!  I've got "dpg-view" on my Linux box (which displays "uncompressed X-Faces") and I've managed to compile [un]compface too... but

Etiqueta real: comp.windows.x
Etiqueta predicha: talk.religion.misc
--------------------------------------------------
Texto test:
 In a word, ye

Conclusiones:

- El F1-score no es muy alto (0.50), lo cual es esperado porque este método es bastante simple.

- Los errores que muestra (ej. confundir rec.autos con alt.atheism) tienen sentido porque:

    - El método depende únicamente de la similaridad semántica superficial (bolsa de palabras), sin contexto profundo.

    - Si hay mucho vocabulario común entre categorías distintas, puede equivocarse.

    - En la salida también se ve que en algunos casos acierta, como en talk.politics.mideast.

#### Parte 3
#### Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
#### (f1-score macro) en el conjunto de datos de test.

In [3]:
# Prueba 1: Entrenar un MultinomialNB básico y evaluarlo

from sklearn.naive_bayes import MultinomialNB

# Entrenar MultinomialNB
nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)

# Predicciones
y_pred_nb = nb_model.predict(X_test)

# Evaluación
f1_nb = f1_score(y_test, y_pred_nb, average="macro")
print("F1-score (macro) MultinomialNB:", f1_nb)


F1-score (macro) MultinomialNB: 0.5854345727938506


In [4]:
# Prueba 2: ComplementNB baseline

from sklearn.naive_bayes import ComplementNB

cnb = ComplementNB()
cnb.fit(X_train, y_train)

y_pred_cnb = cnb.predict(X_test)
f1_cnb = f1_score(y_test, y_pred_cnb, average='macro')
print("F1-score (macro) ComplementNB:", f1_cnb)


F1-score (macro) ComplementNB: 0.692953349950875


In [5]:
# Prueba 3: Grid Search para Naive Bayes

from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

# Creamos pipeline: vectorizador + modelo
pipeline_cnb = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words='english')),
    ('cnb', ComplementNB())
])

# Definimos rejilla de parámetros
param_grid = {
    'tfidf__ngram_range': [(1,1), (1,2)],   # unigramas y bigramas
    'tfidf__min_df': [1, 3, 5],             # frecuencia mínima de términos
    'cnb__alpha': [0.1, 0.5, 1.0]           # suavizado
}

# Grid search con F1 macro
grid = GridSearchCV(pipeline_cnb, param_grid, scoring='f1_macro', cv=3, n_jobs=-1)
grid.fit(newsgroups_train.data, newsgroups_train.target)

print("Mejores parámetros:", grid.best_params_)
print("Mejor F1-score (validación):", grid.best_score_)

# Evaluamos en el test final
best_model = grid.best_estimator_
y_pred_best = best_model.predict(newsgroups_test.data)
print("F1-score (macro) en test con mejor modelo:", f1_score(newsgroups_test.target, y_pred_best, average='macro'))


Mejores parámetros: {'cnb__alpha': 0.1, 'tfidf__min_df': 1, 'tfidf__ngram_range': (1, 2)}
Mejor F1-score (validación): 0.7666500125099275
F1-score (macro) en test con mejor modelo: 0.7097526747834789


Conclusion:

- Con MultinomialNB se obtuvo un F1-score de 0.585

- Con ComplementNB se obtuvo un F1-score de 0.693

- Con ComplementNB optimizado se obtuvo un F1-score de 0.710 en test, maximizando asi el desempeño.

#### Parte 4
#### Transponer la matriz documento–término para estudiar la similaridad entre palabras

In [6]:
# Vectorizamos de nuevo (usamos stopwords y min_df>2 para evitar palabras raras)
vectorizer = TfidfVectorizer(stop_words="english", min_df=3)
X = vectorizer.fit_transform(newsgroups_train.data)

# Matriz término-documento en forma dispersa (NO convertimos a array denso)
X_words = X.T  # sigue siendo sparse

# Diccionario de palabras
terms = vectorizer.get_feature_names_out()

In [7]:
# palabras a analizar
words_to_check = ["god", "windows", "car", "space", "game"]

for word in words_to_check:
    if word in terms:
        idx = np.where(terms == word)[0][0]
        # calculamos similaridad SOLO de esa palabra con todas las demás
        sim_scores = cosine_similarity(X_words[idx], X_words).flatten()
        # ordenamos y tomamos las 5 más similares
        top5_idx = sim_scores.argsort()[::-1][1:6]
        print(f"\nPalabra: {word}")
        for i in top5_idx:
            print(f"   {terms[i]} ({sim_scores[i]:.3f})")
    else:
        print(f"\nPalabra '{word}' no encontrada en el vocabulario")


Palabra: god
   jesus (0.277)
   bible (0.273)
   christ (0.267)
   faith (0.258)
   existence (0.253)

Palabra: windows
   dos (0.307)
   ms (0.227)
   microsoft (0.211)
   nt (0.202)
   file (0.192)

Palabra: car
   cars (0.193)
   dealer (0.176)
   civic (0.170)
   owner (0.155)
   loan (0.154)

Palabra: space
   nasa (0.325)
   shuttle (0.283)
   seds (0.279)
   enfant (0.264)
   exploration (0.237)

Palabra: game
   games (0.213)
   espn (0.187)
   hockey (0.182)
   team (0.181)
   scored (0.179)


Conclusion:

Los resultados tienen mucho sentido semántico:

- god → jesus, bible, christ, faith → coherente con religión.

- windows → dos, ms, microsoft, nt → todo del ecosistema Microsoft.

- car → cars, dealer, civic, owner → coherente con autos.

- space → nasa, shuttle, exploration → relacionado con exploración espacial.

- game → games, espn, hockey, team → vinculado al deporte y juegos.